In [71]:
import os
import json
import pandas as pd
import traceback


In [72]:
from langchain_groq import ChatGroq

In [73]:
pip install python-env

In [74]:
from dotenv import load_dotenv
import os

In [75]:
load_dotenv()

True

In [76]:
KEY=os.getenv("GROQ_API_KEY")

In [77]:
llm=ChatGroq(groq_api_key=KEY,model_name="Gemma-7b-it")

In [78]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026117BD5610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026117BD5A30>, model_name='Gemma-7b-it')

In [79]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [80]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [81]:
TEMPLATE="""
Text: {text}

You are an expert MCQ creator. Based on the above text, your task is to create a quiz consisting of exactly {number} multiple-choice questions for {subject} students in a {tone} tone. 

Guidelines:
1. Ensure all questions are unique and directly related to the provided text.
2. Verify that all questions and answers conform to the information in the text.
3. Use the response  format provided below as a guide.
4. The response should be in a json format to convert it in dataframe.
5. Follow the same{response_json} format.


###RESPONSE_FORMAT
   {response_json}

"""

In [82]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [83]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [84]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [85]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [86]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [87]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [88]:
file_path=r"C:\Users\ACER\mcqq\data.txt"

In [89]:
file_path

'C:\\Users\\ACER\\mcqq\\data.txt'

In [90]:
with open(file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()

In [91]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other s

In [92]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [93]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [94]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON,indent=4)
        }
        )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions

In [95]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2708
Prompt Tokens:1854
Completion Tokens:854
Total Cost:0.0


In [96]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] 

In [97]:
quiz=response.get("quiz")

In [98]:
import json

quiz_json = json.dumps(quiz, indent=4)  # Convert the dictionary to a pretty JSON string
type(quiz_json)


str

In [99]:
import json

# The 'quiz' JSON string from the response
quiz_json = response['quiz']#response.get("quiz")

# Step 1: Parse the JSON string into a Python dictionary
try:
    quiz = json.loads(quiz_json.strip('```json').strip('```'))
    print(type(quiz))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    quiz = {}

# Step 2: Preprocess the dictionary into a list of records
quiz_records = []
if isinstance(quiz, dict):
    for question_id, question_data in quiz.items():
        flat_question = {
            "question_id": question_id,
            "mcq": question_data["mcq"],
            "correct": question_data["correct"]
        }
        # Add options with flattened keys
        flat_question.update({
            f"option_{key}": value for key, value in question_data["options"].items()
        })
        quiz_records.append(flat_question)
else:
    print("Parsed 'quiz' is not a dictionary.")

# Step 3: Print the processed records for verification
for record in quiz_records:
    print(record)


<class 'dict'>
{'question_id': '1', 'mcq': 'What is the primary focus of biology?', 'correct': 'b', 'option_a': 'Study of inanimate objects', 'option_b': 'Study of life forms', 'option_c': 'Study of physical laws', 'option_d': 'Study of mathematical concepts'}
{'question_id': '2', 'mcq': 'Which unifying theme connects all organisms?', 'correct': 'b', 'option_a': 'Their ability to fly', 'option_b': 'Their genetic material', 'option_c': 'Their physical appearance', 'option_d': 'Their speed of growth'}
{'question_id': '3', 'mcq': 'What is a major energy-related process in life?', 'correct': 'a', 'option_a': 'Photosynthesis', 'option_b': 'Cellular respiration', 'option_c': 'Fermentation', 'option_d': 'Combustion'}
{'question_id': '4', 'mcq': 'Which level of organization does biology study?', 'correct': 'd', 'option_a': 'Molecular level', 'option_b': 'Cellular level', 'option_c': 'Tissue level', 'option_d': 'All of the above'}
{'question_id': '5', 'mcq': 'When did life emerge on Earth?', 'c

In [100]:
# Step 4: Save to CSV (optional)
import pandas as pd

# Assuming 'quiz_records' is a list of dictionaries from the previous code
df = pd.DataFrame(quiz_records)

# Step 2: Display the DataFrame to verify the results
df.head()


,question_id,mcq,correct,option_a,option_b,option_c,option_d
0,1,What is the primary focus of biology?,b,Study of inanimate objects,Study of life forms,Study of physical laws,Study of mathematical concepts
1,2,Which unifying theme connects all organisms?,b,Their ability to fly,Their genetic material,Their physical appearance,Their speed of growth
2,3,What is a major energy-related process in life?,a,Photosynthesis,Cellular respiration,Fermentation,Combustion
3,4,Which level of organization does biology study?,d,Molecular level,Cellular level,Tissue level,All of the above
4,5,When did life emerge on Earth?,b,Less than 1 billion years ago,Around 3.7 billion years ago,More than 5 billion years ago,About 100 million years ago


In [101]:
# Step 3: Optionally, save the DataFrame to a CSV file if needed
df.to_csv('quiz_records.csv', index=False)
